In [ ]:
pip install transformers

In [ ]:
# pip install torchvision 

In [ ]:
pip install -U matplotlib

In [ ]:
# pip install tensorflow

In [ ]:
!pip list | grep "tensorflow"   # Check tensorflow==2.0.0, tensorflow-gpu==2.0.0
!pip list | grep "transformers" # Check transformers>=2.0.0

In [ ]:
# pip install tensorflow

In [ ]:
# !pip install --upgrade tensorflow-gpu

In [ ]:
pip install -U scikit-learn

In [ ]:
pip install pandas

In [ ]:
import os 

dir_name= os.getcwd()+'/data/'

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification,TFBertModel
from transformers import InputExample, InputFeatures


model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=3)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

In [ ]:
# nasdaq_files="/userhome/cs/wuxue/Sentiment_Analysis/stock_ticker_datasets/nasdaq.csv"
# nyse_files="/userhome/cs/wuxue/Sentiment_Analysis/stock_ticker_datasets/nyse.csv"
# nasdaq=pd.read_csv(nasdaq_files) 
# nyse=pd.read_csv(nyse_files)

# nasdaq['Symbol']=nasdaq['Symbol'].astype(str)
# nasdaq_input=nasdaq['Symbol']

# nyse['Symbol']=nyse['Symbol'].astype(str)
# nyse_input=nyse['Symbol']
# # 3081
# n = 100  #chunk row size
# nasdaq_df = [nasdaq_input[i:i+n] for i in range(0,nasdaq_input.shape[0],n)]
# nyse_df = [nyse_input[i:i+n] for i in range(0,nyse_input.shape[0],n)]
# df=nasdaq_df+nyse_df

In [ ]:
# df = pd.DataFrame()
# for list in nasdaq_df:
#      for name in list:
#             try:
#                 df_tweets = pd.read_csv('/userhome/cs/wuxue/Sentiment_Analysis/data-tweets/data-$%s-tweets.csv'%name ,names=['dates','tweets'],index_col='dates')
#                 df_dates=pd.read_csv('/userhome/cs/wuxue/Sentiment_Analysis/stock_label/data-%s-label.csv'%name,names=['dates','label'],index_col='dates')

#                 merge=pd.merge(df_tweets,df_dates, how='inner', left_index=True, right_index=True)
#                 df=df.append(merge)
                
#             except:
#                 print('no such file exist')
#                 pass
# df.to_csv('labelled_tweets.csv')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def read_tweets(name):
   
    print('reading tweets..')
    
    df = pd.read_csv(name)
    print(df)

    df = df.drop(['Unnamed: 0','dates','ticker','compound_vader_score'],axis=1)
    df=df.reset_index(drop=True)
    df=df.dropna()
    
    possible_labels = df.pred_label.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    
    
    df['pred_label'] = df.pred_label.replace(label_dict)
    x_train,y_train=train_test_split(df,test_size=0.2,random_state=0,stratify=df.pred_label.values)
#     print(x_train)
    return x_train,y_train
# read_tweets('nasdaq_labelled_tweets.csv')

def read_news(name):
    print('reading news..')
    
    df = pd.read_csv(name)
    print(df)
   
    df = df.drop(['Unnamed: 0','dates','ticker','compound_vader_score'],axis=1)
    df=df.reset_index(drop=True)
    df=df.dropna()
#     label_dict = {0:1,1:2,2:3}
    possible_labels = df.pred_label.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    print(label_dict)

    df['pred_label'] = df.pred_label.replace(label_dict)
    x_train,y_train=train_test_split(df,
                                     test_size=0.2,
                                     random_state=0,
                                     stratify=df.pred_label.values)
#     print(x_train)
    return x_train,y_train

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
    print('converting data')
    
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
    return train_InputExamples, validation_InputExamples

# train_InputExamples, validation_InputExamples = convert_data_to_examples(x_train,  y_train, 'tweets_x', 'tweets_y')
  


In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    print('converting examples to tf_datasets')
    features = [] # -> will hold InputFeatures to be converted later
    
    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )
    def gen():
        
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )



In [ ]:
def model_train(file_name):
    print('model training')
    x_train,y_train=read_tweets(file_name)    

    train_InputExamples, validation_InputExamples = convert_data_to_examples(x_train, y_train, 'tweets', 'pred_label')
#     print(train_InputExamples)
    train_data = convert_examples_to_tf_dataset(train_InputExamples.values.tolist(), tokenizer)
    print(train_data)
    train_data = train_data.shuffle(100).batch(15).repeat(2)


    validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
#     validation_data = convert_examples_to_tf_dataset(validation_InputExamples.values.tolist(), tokenizer)
    validation_data = validation_data.batch(15)
    return train_data,validation_data

def news_model_train(file_name):
    print('model training')
    x_train,y_train=read_news(file_name)    

    train_InputExamples, validation_InputExamples = convert_data_to_examples(x_train, y_train, 'news', 'pred_label')
#     print(train_InputExamples)
    train_data = convert_examples_to_tf_dataset(train_InputExamples.values.tolist(), tokenizer)
    print(train_data)
    train_data = train_data.shuffle(100).batch(15).repeat(2)


    validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
#     validation_data = convert_examples_to_tf_dataset(validation_InputExamples.values.tolist(), tokenizer)
    validation_data = validation_data.batch(15)
    return train_data,validation_data
# model_train('nasdaq_labelled_tweets.csv')

In [ ]:
def model_validation(train_data,validation_data):
    print('validation..')
#     model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    

    # model.fit(train_data, epochs=2, validation_data=validation_data)
    history=model.fit(train_data, epochs=2, validation_data=validation_data)
    print(history)
    return history

In [ ]:
import matplotlib.pyplot as plt
import os
from datetime import datetime

def accuracy_visualization(history):
    print('visualization')
    loss_train = history.history['accuracy']
    loss_train = np.array(loss_train)
    loss_val = history.history['val_accuracy']
    loss_val=np.array(loss_val)
    print(loss_train)
    print(loss_val)
    epochs=range(1, len(loss_train) + 1)
    plt.plot(epochs, loss_train, 'g', label='Training accuracy')
    plt.plot(epochs, loss_val, 'b', label='validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(ymin=0)
    plt.ylim(ymax=1)
    plt.legend()
    plt.show()


In [ ]:
# data training
def start(file_name):
    train_data,validation_data=model_train(file_name)
    history=model_validation(train_data,validation_data)
    return history
    

def news_start(file_name):
    train_data,validation_data=news_model_train(file_name)
    history=model_validation(train_data,validation_data)
    return history
    

In [ ]:
nasdaq_vader_tweets_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_vader_tweets.csv')
nasdaq_tweets_history=start(nasdaq_vader_tweets_path)


In [ ]:
accuracy_visualization(nasdaq_tweets_history)

In [ ]:
nasdaq_vader_news_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_vader_news.csv')
    
nasdaq_news_history=news_start(nasdaq_vader_news_path)



In [ ]:
accuracy_visualization(nasdaq_news_history)

In [ ]:
nyse_vader_news_path=os.path.join(dir_name,'train-data/nyse/nyse_vader_news.csv')
nyse_news_history=news_start(nyse_vader_news_path)


In [ ]:
accuracy_visualization(nyse_news_history)

In [ ]:
nyse_vader_tweets_path=os.path.join(dir_name,'train-data/nyse/nyse_vader_tweets.csv')
nyse_tweets_history=start(nyse_vader_tweets_path)


In [ ]:
accuracy_visualization(nyse_tweets_history)

In [ ]:

hkex_news_history=os.path.join(dir_name,'train-data/hkex/hkex_vader_news.csv')
hkex_news_history=news_start(hkex_news_history)




In [ ]:
accuracy_visualization(hkex_news_history)